![example](images/top_image0.jpg)

Copyright: <a href='https://www.123rf.com/profile_tomkli'>Thomas Klinder</a>


# <center>Increase Your Property Value and Stretch Your House Buying Budget Further</center>

<span style="color:red; font-size:1.5em">**Phase 2 Final Project**</span><br>
* Student name: <b>Elena Kazakova</b>
* Student pace: <b>full time</b>
* Cohort: <b>DS02222021</b>
* Scheduled project review date/time: <span style="color:red"><b>TBD</b></span>
* Instructor name: <b>James Irving</b>
* Blog post URL: <span style="color:red"><b>TBD</b></span>

## Table of Contents 

*Click to jump to matching Markdown Header.*<br>

- **[Introduction](#INTRODUCTION)<br>**
- **[Obtain](#Obtain)**<br>
- **[Scrub](#Scrub)**<br>
- **[Explore](#Explore)**<br>
- **[Model](#Model)**<br>
- **[iNterpret](#iNterpret)**<br>
- **[Conclusions/Recommendations](#Conclusions-and-Recommendation)<br>**
___

# Introduction

## Business Problem
This project is the Inference Analysis project of King County, WA house prices, and various factors that might affect the sales price.<br>
This study aims to build a model(s) of house sale prices depending on the features of the property in the dataset provided. This information can be helpful for house owners, house buyers, and real estate agents in the county.

# Obtain

## Data Understanding

The dataset used in this project has been downloaded from [KAGGLE site]( https://www.kaggle.com/harlfoxem/housesalesprediction?select=kc_house_data.csv). The dataset includes the information about properties sold in King County of Washington State between May 2014 and May 2015. The area consists of Seattle city area but does not include the inner city. The dataset consists of 21 dependent and independent variables and 21597 records.

### Importing Python tools and utilities

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import folium
import json

import plotly.express as px

import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.stats.api as sms
import sklearn.metrics as metrics
#import plotly.graph_objects as go

import scipy.stats as stats

import math
#import pickle
import scipy.stats

from matplotlib import style

from statsmodels.formula.api import ols
from statsmodels.stats.outliers_influence import variance_inflation_factor

#from pandasql import sqldf

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
#from sklearn.preprocessing import OneHotEncoder, PolynomialFeatures, StandardScaler, RobustScaler, MinMaxScaler
from sklearn.preprocessing import OneHotEncoder, PolynomialFeatures, RobustScaler, MinMaxScaler

from sklearn.feature_selection import RFE
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_score
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

from scipy.stats import pearsonr
from IPython.display import display, Math, Latex
from folium import plugins
from plotly.subplots import make_subplots

#from mlxtend.evaluate import bias_variance_decomp

from warnings import filterwarnings
filterwarnings('ignore')

%matplotlib inline

### Functions used

In [ ]:
def count_unique_records(df):
    unique_records=[]
    for column in df.columns:
        n = df[column].nunique()
        unique_records.append((column,n))
    print(unique_records)
    return None

def count_dups_field(field1):
    dups = df.pivot_table(index = [field1], aggfunc ='size')
    return dups

def count_dups_fields(field1, field2):
    dups = df.pivot_table(index = [field1, field2], aggfunc ='size')
    return dups

def remove_columns(df, y_columns=['price'], x_columns=[], exclude_columns=[], add_constant=True):
    
    if x_columns==[]:
        x_columns=list(df.drop(columns=y_columns, axis=1))
        
    [x_columns.remove(columns) for columns in exclude_columns]
    
    df_x=df[x_columns]
    df_y=df[y_columns]
    
    return df_x, df_y
   
# Function to convert geo coordinates to distance from center. I am using coordinates
def distance_from_center(lat_coord,lon_coord):
    R = 3959.999

# I am using geo coordinates of Seattle 47.6062° N, 122.3321° W, from Wikipedia

    lat1 = math.radians(47.6062)
    lon1 = math.radians(122.3321)
    lat2 = math.radians(lat_coord)
    lon2 = math.radians(lon_coord)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    d=R*c
    return round(d,1)

def jointplot(df):
    sns.set_style('white')
    for col in df.columns:
        g=sns.jointplot(x=col, y='price', data=df, size=5, kind='reg', marginal_ticks=True,
                        joint_kws={'line_kws':{'color':'green'}}, height=15, space=0.7)
        name=col
        R2,p= scipy.stats.pearsonr(x=df[col], y=df.price)
        g.fig.suptitle('For {}: R2 coefficient is {}, p-value is {}'.format(name, round(R2,4),p))
        g.fig.tight_layout()
        g.fig.subplots_adjust(top=0.85)
        
    return None

def r2_p(df):
    for col in df.columns:
        name=col
        R2,p= scipy.stats.pearsonr(x=df[col], y=df.price)
        print('For {}: R2 coefficient is {}, p-value is {}'.format(name, round(R2,4),p))
    return None

# This is a snippet from https://www.analyticsvidhya.com/blog/2020/03/what-is-multicollinearity/
def calc_vif(X):

    # Calculating VIF
    vif = pd.DataFrame()
    vif["variables"] = X.columns
    vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

    return(vif)

# This is a snippet from https://atmamani.github.io/cheatsheets/seaborn/seaborn_cheat_sheet_1/
def distribution(column):
    col_mean = column.mean()
    col_sd = column.std()
    skew_val = stats.skew(column, bias=False)
    kurt_val = stats.kurtosis(column,bias=False)
    
    ax = sns.distplot(column, kde_kws={"color": "r", "lw": 2, "label": "KDE", "bw_adjust": 3})
        
    ax.axvline(x=col_mean, color='black', linestyle='dashed')

    ax.axvline(x=col_mean + col_sd, color='red', linestyle='dotted')
    ax.axvline(x=col_mean - col_sd, color='red', linestyle='dotted')

    ax.set_title('$\mu = {}$ | $\sigma = {}$ | Skew = {} | Kurtosis = {}'.
                 format(round(col_mean, 2), round(col_sd, 2), round(skew_val,2), round(kurt_val,2)))
    
    plt.subplots_adjust(top=0.5)
    plt.tight_layout()
    
    return None

def boxen_plot(df,colname):
    ax = sns.catplot(x=df[colname], y=df.price/1000000, kind="boxen",
                 data=df.sort_values(colname), height=7, aspect=8/5)
    ax.set_xticklabels(fontsize=12)
    ax.set_yticklabels(fontsize=12)
    plt.ylabel('Price in millions', fontsize=15)
    plt.xlabel(colname,fontsize=15)
    plt.grid()
    plt.show()
    return None

#Zipcode choropleth maps with average values per a zipcode (King County)
def map_choropleth_zip(df, column, title, column_name):
    fig=px.choropleth_mapbox(data_frame=df, locations='zipcode', geojson=KC_zip_json, color=column, 
                         mapbox_style='open-street-map', zoom=8.5, height=900, featureidkey='properties.ZCTA5CE10', 
                        center={'lat': 47.403768, 'lon': -122.005863}, opacity=0.4,
                        color_continuous_scale=px.colors.sequential.YlOrRd,
                        title=title,
                        template = "plotly_dark", 
                        labels={
                            column: column_name})
    fig.update_layout(
    font_family="Arial",
    font_size=16,
    font_color="white",
    title_font_family="Arial",
    title_font_color="white",
    title_font_size=20)
    
    fig.update_layout(
    title={
        'y':0.98,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top',
    })
    
    fig.show()
    return None


### Importing data

In [ ]:
# Importing raw data
df=pd.read_csv('data/kc_house_data.csv')
pd.set_option('display.width', 1000)
df.head()

In [ ]:
df.info()

In [ ]:
# Displaying tuples of fields and unique records in them
count_unique_records(df)

In [ ]:
# Finding all duplicate IDs

df_a=pd.DataFrame(count_dups_field('id'))
df_a.columns=['count']
df_a = df_a[df_a['count'] > 1].sort_values('count', ascending=False)
df_a.reset_index(level=0, inplace=True)
df_a

In [ ]:
# Listing duplicate IDs and associated dataset records to find out the reason they are in the file twice

df_dup=pd.DataFrame()
for i in range(len(df_a)):
    df_dup=df_dup.append(df[df.id==df_a.id[i]], ignore_index=True)
df_dup

<div class="alert alert-block alert-danger">The inspection of the records shows that the records with duplicate IDs have different sale dates and sale prices. However, all other features remain the same. Records with the same set of predictors but different prices would introduce additional "noise" to the data. Because there are only 353 records with that problem, I decided to drop them from the dataset.</div>

In [ ]:
#listing datatype, number of null values, min and max values in the fields of the dataset

fields1=['bedrooms','bathrooms','floors','waterfront', 'view','condition','grade']
fields2=['sqft_lot15', 'sqft_living15','yr_renovated','yr_built','sqft_above','sqft_living','sqft_lot']
for column in df.columns:
    type_=df[column].dtypes
    num_nulls=df[column].isna().sum()
    min_=0
    max_=0
    unique_=[0]
    if column in fields1:
        unique_=df[column].unique()
        unique_.sort()
    else:
        if column in fields2:
            min_=df[column].min()
            max_=df[column].max()
        else:
            continue
    print('Column name:', column)
    print('Type:', type_)
    print('Number of null values', num_nulls)
    print('Unique values:',unique_)
    print('Min value: ', min_, 'Max value:', max_)
    print('***********************************')

### Description of the fields

The file has 21597 records with 21 columns, out of which 11 columns have integer values, 8 are real numbers, and 2 are strings.<br>
    The annotation to the fields and associated data<br> (link to the definitions [here](https://github.com/emilyageller/king_county_house_prices))
* <b>id:</b> Unique ID for each home sold

    1. no NULL values<br>
    2.integer numbers<br>
    3.176 duplicate records
><span style="color:red"><b>353 rows to be dropped</b></span>

* **date:** Date of the sale
>no NULL values<br>
string<br>

><span style="color:red"><b>Convert to DateTime type</b></span>

* **price** - Price of each home sold
>no NULL values<br>
Real numbers<br>
Minimum price: 78000<br>
Maximum price: 7700000

* **bedrooms** - Number of bedrooms
>no NULL values<br>
>Integer numbers, between 1 and 33

* **bathrooms** - Number of bathrooms, where .5 accounts for a room with a toilet but no shower
>no NULL values<br>
>Real numbers, between 0.5 and 8.0

* **sqft_living** - Square footage of the house interior living space
>No NULL values<br>
Integer numbers<br>
Minimum value: 370<br>
Maximum value: 13540

* **sqft_lot** - Square footage of the land lot
>No NULL values<br>
Integer numbers<br>
Minimum value: 520<br>
Maximum value: 1651359

* **floors** - Number of floors
>no NULL values<br>
>Real numbers, between 1.0 and 3.5

* **waterfront** - A categorical variable for whether the house was overlooking the waterfront or not
>**2376** NULL values<br>
>Real numbers, only two values 1.0 and 0.0<br>

><span style="color:red"><b>Convert to a categorical variable<br>
Waterfront, not Waterfront</b></span><br><br>
><span style="color:red"><b>Replace NULL values with "Missing" category</b></span>

* **view** - A categorical variable describing how good the view of the property was
>**63** NULL values<br>
>Real numbers: 1.0, 2.0, 3.0, 4.0<br>

><span style="color:red"><b>Convert to a categorical variable<br>
Poor, Fair, Good Excellent</b></span>

><span style="color:red"><b>Replace NULL values with "Missing" category</b></span>

* **condition** - A categorical variable describing the condition of the house
>no NULL values<br>
>Integer numbers, between 1 and 5<br>

><span style="color:red"><b>Convert to a categorical variable<br>
Poor, Fair, Good, Very Good, Excellent</b></span>

* **grade** - A categorical variable describing the quality of construction, from 1 to 13; 1-3 falls short of building construction and design, 7 has an average level of construction and design, and 11-13 have a high quality level of construction and design.
>no NULL values<br>
>Integer numbers, between 3 and 13<br>

* **sqft_above** - The square footage of the interior housing space that is above ground level
>No NULL values<br>
Integer numbers<br>
Minimum value: 370<br>
Maximum value: 9410

* **sqft_basement** - The square footage of the interior housing space that is below ground level
>No NULL values<br>
String<br>

><span style="color:red"><b>Convert to integer</b></span><br>

* **yr_built** - The year the house was initially built
>No NULL values<br>
Integer numbers<br>
Minimum value: 1900<br>
Maximum value: 2015

* **yr_renovated** - The year of the last house renovation
>**3842** NULL values<br>
>Real numbers, between 0.0 and 2015.0<br>

><span style="color:red"><b>Convert to integer</b></span><br>

* **zipcode** - What zipcode area the house is in
>no NULL values<br>
Integer numbers, 70 unique values<br>

><span style="color:red"><b>Convert to categorical variable or drop</b></span>

* **lat** - Lattitude
>no NULL values<br>
>Real numbers

* **long** - Longitude
>no NULL values<br>
>Real numbers

* **sqft_living15** - The square footage of interior housing living space for the nearest 15 neighbors
>No NULL values<br>
Integer numbers<br>
Minimum value: 399<br>
Maximum value: 6210

* **sqft_lot15** - The square footage of the land lots of the nearest 15 neighbors
>No NULL values<br>
Integer numbers<br>
Minimum value: 651<br>
Maximum value: 871200

### Initial cleaning of the data

In [ ]:
df.info()

#### 1. Dropping duplicate rows for houses sold twice in the timeframe of the dataset

In [ ]:
#Dropping duplicate rows for houses sold twice in the timeframe of the dataset
df = df.drop_duplicates(subset='id', keep="first")
df

#### 2. Converting the 'date' field to DateTime formate and making sure it worked

In [ ]:
# date string to datetime type
df['date'] = pd.to_datetime(df['date'])  

In [ ]:
#Checking if the conversion went OK
mask = (df['date'] > '9/24/2014') & (df['date'] <= '4/22/2015')
df.loc[mask]

#### 3. Locations of the houses with missing 'waterfront' values

In [ ]:
# Possible strategies:
# 1. Check if there are waterfront properties among neighbors within a certain distance range
# 2. Make a map and place properties with missing values on it visually
# 3. What is the longitude of the bay shore? Any house with a missing value too far away from it 
#    should have their waterfront value set to 0. Hopefully, it will eliminate most of the missing values in this field

In [ ]:
# Leaving just coordinates of the waterfront NaN properties
mask = (df['waterfront'].isna())
df_wf_na_coord=df.loc[mask]
df_wf_na_coord.drop(df_wf_na_coord.columns[[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,19,20]], axis=1, inplace=True)

df_wf_na_coord

##### 3.1 Visual assesment of the houses with Null value in the 'waterfront column

In [ ]:
# Initializing the map based on the Wikipedia location of Seattle
KCMap = folium.Map(location=[47.6171,-122.3249],  tiles='Stamen Terrain', zoom_start=9)

# for each row in the KC_house dataset with missing waterfront value, 
# plot the corresponding latitude and longitude on the map
for index, row in df_wf_na_coord.iterrows():
    folium.CircleMarker((row['lat'], row['long']), radius=1, weight=2, 
                        color='red', fill_color='red', fill_opacity=.5).add_to(KCMap)
    
display(KCMap)
# adding the heatmap
KCMap.add_child(plugins.HeatMap(data=df_wf_na_coord[['lat', 'long']], radius=20, blur=10))


In [ ]:
# Displaying
# the percentage of waterfront properties in the dataset with missing waterfront values,
# the percentage of waterfront properties in the full dataset
# The systemic error introduced to the full dataset if missing waterfront values would be replaced with 0

a=round((20/len(df_wf_na_coord))*100,3)

b=round((len(df[df['waterfront'] == 1])/len(df)*100),2)

c=round(round(((len(df[df['waterfront'] == 1])+20)/len(df)*100),3)-round((len(df[df['waterfront'] == 1])/len(df)*100),3),3)

print('{}%, {}%, {}%'.format(a, b, c))

<div class="alert alert-block alert-danger">It is self-evident from the visuals above that the vast majority of the houses are located inland. Simple zooming in the maps allows a rough counting of alleged waterfront properties. The estimate is approximately 20 waterfront houses. It is 0.85% of all properties with no value in 'waterfront column (2353). In the primary dataset, the percentage of waterfront properties out of the total number of properties is 0.68%. The numbers above indicate that replacing the NaN values with 0 would introduce a systemic error of 0.01% to the whole system.<br>
    <b>Conclusion:</b> The NULL values in the 'waterfront' column will be replaced with 0.
    </div>

##### 3.2 Replacing NULL values in 'waterfront' column and converting the column to the integer datatype to make it categorical

In [ ]:
#Replacing NaN values in 'waterfront' column

df.loc[df.waterfront.isna(),'waterfront']=0
df.waterfront=df.waterfront.astype('int64')


subset_df = df[df['waterfront'] == 1]
count = len(subset_df)
print(count)
#df.info()

In [ ]:
df.info()

##### 3.3 Replacing NULL values in waterfront  and view field  using IterativeImputer

In [ ]:
# Dropping columns with str datatypes to use IterativeImputer on the rest

df_to_II=df.drop(['date','id','zipcode','sqft_basement'],axis=1)
df_to_II.info()

In [ ]:
# Using IterativeImputer to fill in Nan cells

imp = IterativeImputer(max_iter=10,random_state=0)
imp.fit(df_to_II)
imputed_df_to_II = imp.transform(df_to_II)
imputed_df = pd.DataFrame(imputed_df_to_II, columns=df_to_II.columns)
imputed_df.info()

In [ ]:
df.head(20)

In [ ]:
imputed_df.head(20)

In [ ]:
# Creating a subset of records with imputed NaN values

subset_df = imputed_df[(imputed_df['yr_renovated'] < 1934) & (imputed_df['yr_renovated'] != 0)]
count = len(subset_df)
print(count)

In [ ]:
# Listing Imputed NaN values in yr_renovated column
list_unique=list(df.yr_renovated.unique())

inverse_boolean_series = ~imputed_df.yr_renovated.isin(list_unique)
inverse_filtered_df = imputed_df[inverse_boolean_series]
inverse_filtered_df.yr_renovated.sort_values()

In [ ]:
# Listing Imputed NaN values in waterfront column
inverse_boolean_series = ~imputed_df.waterfront.isin([0,1])
inverse_filtered_df = imputed_df[inverse_boolean_series]
inverse_filtered_df.waterfront.sort_values() 


In [ ]:
# Listing Imputed NaN values in view column
inverse_boolean_series = ~imputed_df.view.isin([0,1,2,3,4])
inverse_filtered_df = imputed_df[inverse_boolean_series]
inverse_filtered_df.view.sort_values() 

<div class="alert alert-block alert-danger">
    <b>Conclusion:</b>  Based on the results of IterativeImputer the original approach of replacing missing values in 'waterfront', 'yr_renovated', and 'view'  with 0 will be taken<br>    </div>

#### 4. Replacing NULL values in yr_renovated column with 0. value and changing the type to integer

In [ ]:
df.loc[df.yr_renovated.isna(),'yr_renovated']=0.0
df.yr_renovated=df.yr_renovated.astype('int64')

#### 5. Replacing '?' values in 'sqft_basement' column with 0. value and changing the type to integer

In [ ]:
df.loc[(df.sqft_basement == '?')]

In [ ]:
# Converting '?' sqft_basement values to 0 and listing unique values
df.loc[(df.sqft_basement == '?'),'sqft_basement']='0.0'

df.sqft_basement=df.sqft_basement.astype('float64')

df.sqft_basement=df.sqft_basement.astype('int64')

#Checking the result
df_temp=df.sqft_basement.unique()
list1 = df_temp.tolist()
list1.sort()
print(list1)

#### 6. Replacing NULL values in 'view' column with 0.0 and making it integer

In [ ]:
df.loc[df.view.isna(),'view']=0
df.view=df.view.astype('int64')

df.info()

# Scrub and Explore

## Additional data cleaning

### Dropping non-needed fields

In [ ]:
# Dropping 'zipcode' variable because I think there are better indicators of a location.
# Zipcodes boundaries are usually drawn out of convenience for postal services or
# other more formal reasons than geographic location

# Dropping 'id' field
# Resettingh index because of the removal of the duplicates

df_1=df
df_1=df_1.drop(['id','zipcode'], axis=1)
#df_1.tail()


df_1.reset_index()

<div class="alert alert-block alert-success">
    <b>21420</b> records out of the original <b>21597</b> left
    </div>


## Exploring distributions and correlation of original variables

### Numerical variables: Investigating distributions and correlations between the original, minimally processed predictors and the target (price) 

#### Histograms and pair correlations of the original predictor variables

In [ ]:
df_1.info()

In [ ]:
# Investigating histograms

df_1.hist(figsize=(20, 30), bins='auto')

**Based on the histograms above**
> 1. The following variables should be considered categorical:
>>Waterfront<br>
Condition<br>
View<br>
> 2. sqft_basement, sqft_lot, sqft_lot15, and yr_renovated have a large number of zeros and are strong candidates for removal of outliers and/or engineered variables<br>
> 3. Latitudes and Longitudes can be used as descriptors of a geographic location of a property. However, I think there is  a better variable to describe the location of a property, a distance from the center of the city, which can be calculated from geocoordinates.<br>
    >4. The target variable, the price of the property, has a strong positive skew attributed to outliers in the higher price bracket. The strategy is to **remove the outliers and to transform the variable** to make it more normally distributed

In [ ]:
# Checking for correlations between the variables with Pearson coefficient between 1 and 0.3
# I am using the same approach and reusing the code from Lesson 19

df_coeff=df_1.corr().abs().stack().reset_index().sort_values(0, ascending=False)
df_coeff['pairs'] = list(zip(df_coeff.level_0, df_coeff.level_1))
df_coeff.set_index(['pairs'], inplace = True)
df_coeff.drop(columns=['level_1', 'level_0'], inplace = True)
df_coeff.columns = ['cc']
df_coeff.drop_duplicates(inplace=True)
df_coeff[((df_coeff.cc>.3) & (df_coeff.cc <1))]

 **The variables which have the strongest correlations with the price are**<br>
* sqft_living<br>
* grade<br>
* sqft_above<br>
* sqft_living15<br>
* bathrooms<br>
* view<br>
* bedrooms<br>
*lat

#### Establishing an intermediate dataframe and removing some outliers from it

In [ ]:
# Removing extreme values from sqft_lot, sqft_lot15, bathrooms, and price
# On one hand, it can be left to the step of outlier removal, on the other doing it now
# will help a visual investigation of the distributions of the variables mentioned above

df_2=df_1[(df_1.sqft_lot >100) & (df_1.sqft_lot<40000)]

df_2=df_2[(df_2.sqft_lot15 >100) & (df_2.sqft_lot15<40000)]

df_2=df_2[df_2.price < 2500000]

df_2=df_2[df_2.bedrooms < 30]

df_2.info()


In [ ]:
# Creating a new categorical variable: month
df_2['month'] = pd.to_datetime(df_2['date']).dt.month

# Creating a new numerical variable: distance (distance from center)
df_2['distance'] = df_2.apply(lambda row: distance_from_center(row.lat,abs(row.long)), axis=1) 

# Creating a new categorical variable: basement_exists (1/0), integer datatype
df_2.loc[(df_2['sqft_basement'] > 50), 'basement_exists'] = 1
df_2.loc[(df_2['sqft_basement'] <= 50), 'basement_exists'] = 0
df_2.basement_exists=df_2.basement_exists.astype('int64')

In [ ]:
# Creating a new categorical variable (integer datatype) renovation_done with values [0,1,2,3,4]

# 0 representing renovation never done on houses more than 9 years old (yr_built between 2015 and 2006)
# 1 representing renovation done more than or equal 50 years ago
# 2 representing renovation done between 30 and 49 years ago
# 3 representing renovation done between 29 and 10 years ago
# 4 representing renovation done between 9 and 1 year ago OR houses built less or equal 9 years ago
#   (yr_built between 2015 and 2006)

df_2.loc[((df_2['yr_renovated'] == 0) & (df_2['yr_built'] < 2006)), 'renovation_done'] = 0
df_2.loc[((2015-df_2['yr_renovated'] >= 50) & (df_2['yr_renovated'] != 0)), 'renovation_done'] = 1
df_2.loc[((2015-df_2['yr_renovated'] < 50) & (2015-df_2['yr_renovated'] >= 30)), 'renovation_done'] = 2
df_2.loc[((2015-df_2['yr_renovated'] < 30) & (2015-df_2['yr_renovated'] >= 10)), 'renovation_done'] = 3
df_2.loc[((2015-df_2['yr_renovated'] < 10) | (df_2['yr_built'] >= 2006)), 'renovation_done'] = 4

df_2.renovation_done=df_2.renovation_done.astype('int64')

In [ ]:
# Dropping sqft_basement variable
# Dropping latitude & longtitude variable
# Dropping date variable# Resetting index due to removal of records with extreme values in 
# sqft_lot, sqft_lot15, bathrooms and price

df_2=df_2.drop(['sqft_basement','date', 'lat','long','yr_renovated'], axis=1)

df_2=df_2.reset_index(drop=True)

df_2

<div class="alert alert-block alert-success">
    <b>df_2</b> DataFrame<br><br><b>20015</b> records out of the original <b>21597</b> left
    </div><br>
<div class="alert alert-block alert-danger">
    <b>Index reset</b>
    </div><br>

#### Plotting numerical variables against the target variable

In [ ]:
# Dropping categorical variables to simplify the analysis of the numerical ones

df_num1=df_2.drop(['waterfront','view','condition','basement_exists','renovation_done','month'], axis=1)


In [ ]:
# Displaying the Coefficients of Determination and p-values for the remaining variables
r2_p(df_num1)

In [ ]:
# Joint plot of the original (not altered) numerical variables
jointplot(df_num1)

In [ ]:
df_num1.info()

In [ ]:
# Removing extreme values from bedrooms, bathrooms, floors, and distance
# The observations with the extreme values of these variables are visually identifiable 

df_num2=df_num1[(df_num1.bedrooms < 9) & (df_num1.bathrooms < 5.5) &(df_num1.distance < 30) 
               & (df_num1.floors<3.5)]
df_num2.drop(['yr_built'], axis=1, inplace=True)
df_num2.reset_index(drop=True, inplace=True)

In [ ]:
df_num2.info()

<div class="alert alert-block alert-success">
    <b>df_num2</b> DataFrame<br><br><b>19811</b> records out of original <b>21597</b> left
    </div><br>
<div class="alert alert-block alert-danger">
    <b>Index reset</b>
    </div><br>

In [ ]:
# Joint plot of numerical variables after adjustments
jointplot(df_num2)

In [ ]:
r2_p(df_num2)

In [ ]:
df_num2.info()

#### Visualizing numerical predictors correlation with the price and with each other using heat map

In [ ]:
fig, ax = plt.subplots(figsize=(15,10))
sns.heatmap(df_num2.corr(), cmap="cubehelix", annot=True)

In [ ]:
# Bedrooms is relatively highly correlated with bathrooms and sqft_living; correlations of floors, sqft_lot and sqft_lot15 
# with prices are low. Dropping these variables

df_num3=df_num2.drop(['bedrooms','sqft_lot','sqft_lot15','floors'], axis=1)

In [ ]:
fig, ax = plt.subplots(figsize=(15,10))
sns.heatmap(df_num3.corr(), cmap="cubehelix", annot=True)

In [ ]:
r2_p(df_num3)

#### Exploring the distribution of the remaining numerical predictors

In [ ]:
# Plotting KDE and distribution plots of the remaining variables

continuous=['price','sqft_living','sqft_living15','sqft_above','distance']
for col in continuous:
    fig, ax =plt.subplots(figsize=(5, 5))
    distribution(df_num3[col])

discrete=['bathrooms','grade']
for col in discrete:
    fig, ax =plt.subplots(figsize=(5, 5))
    ax = sns.distplot(df_num3[col], bins=10, kde_kws={"color": "r", "lw": 2, "label": "KDE", "bw_adjust": 4})
    skew_val = stats.skew(df_num3[col], bias=False)
    kurt_val = stats.kurtosis(df_num3[col],bias=False)
    print('Column: {} | Skewness = {} | Kurtosis = {}'.
                 format(col, round(skew_val,2), round(kurt_val,2)))

<div class="alert alert-block alert-info">All but one variable (distance) are left shifted, which is indicated by the skewness values >1, with price has the most skewed distribution. Because the skewness it to the left (positive values), the log transformation might be needed to normalize the variables. Kurtosis values for all variables are different from 0 (Pearson's definition of kurtosis of a normal distribution). <br>
    Price distribution is highly <b>Leptokurtic</b>, other variables are slightly <b>Leptokurtic</b> (sqft_living, sqft_above, sqft_living15, grade), slightly <b>Platykurtic</b> (distance) or almost <b>Mesokurtic</b> (bathrooms)</div>


In [ ]:
# Diamond box plots for descrete numerical variables

discrete=['bathrooms','grade']
for col in discrete:

    ax = sns.catplot(x=col, y='price', kind="boxen",
            data=df_num3.sort_values(col), height=9, aspect=12/9)
    ax.set_xticklabels(rotation=45, ha='right', fontsize=12)
    ax.set_yticklabels(fontsize=12)
    plt.ylabel('Price in millions', fontsize=15)
    plt.xlabel(col, fontsize=15);
    plt.grid()
    plt.show()

<div class="alert alert-block alert-info"> Based on the distribution plots, variable 'bathroom' has a symmetrical distribution (Skewness is 0.28), with a very low kurtosis (0.09) indicative of a <b>Mesokurtic</b> curve (Gaussian distribution has a kurtosis of 0 by Pearson's definition used by scipy.stats.kurtosis method)<br>
Variable 'grade' is slightly skewed to the right (0.73) and relatively low kurtosis, slightly above 1.<br>
The pronounced correlation of these variables with the price is identifiable in the box plots above. The plots show that the numbers of outliers in the distribution of the variables are reasonable. Both variables have a wider range of values in the higher price brackets.</div>

### Numerical variables: Exploring Mutual Correlation Coefficients and Variance Inflation Factor

#### Using VIF as an indicator of collinearity between independent variables

In [ ]:
calc_vif(df_num3)

In [ ]:
# Dropping fields that have high VIF. I decided to leave in sqft_living versus sqft_living15 due to the fact that it is
# easier to interpret in a model and it is a feature under control of a property owner (versus sqft_living15 which is
# a charecteristic of a neighborhood)

df_num4=df_num3.drop(['price','sqft_above','sqft_living15'],axis=1)

calc_vif(df_num4)

#### Pearson coefficients analysis of the remaining independent variables

In [ ]:
df_num4['price']=df_num3['price']

In [ ]:
# Displaying R2s for the remaining variables
r2_p(df_num4)

In [ ]:
# Displaying correlations between predictors and the target
jointplot(df_num4)

In [ ]:
# Using heatmap to visualize mutual relationships between predictors and their correlations with the target
fig, ax = plt.subplots(figsize=(15,10))
sns.heatmap(df_num4.corr(), cmap="cubehelix", annot=True)

In [ ]:
# Checking for correlations between the variables with pearson coefficient between 1 and 0.3


df_coeff=df_num4.corr().abs().stack().reset_index().sort_values(0, ascending=False)
df_coeff['pairs'] = list(zip(df_coeff.level_0, df_coeff.level_1))
df_coeff.set_index(['pairs'], inplace = True)
df_coeff.drop(columns=['level_1', 'level_0'], inplace = True)
df_coeff.columns = ['cc']
df_coeff.drop_duplicates(inplace=True)
df_coeff[((df_coeff.cc>.3) & (df_coeff.cc <1))]

<div class="alert alert-block alert-danger">Mutual correlation coefficients between the remaining independent variables are 
slightly higher or below 0.7. I am leaving sqft_living, grade, and bathroom variables in because of their logical connection with a property price despite their multicollinearity (0.74 & 0.73 are above the 0.7 threshold).<br>

Therefore the remaining numerical variables for <u>modeling</u> are<br><br>
<b>1. grade<br>
2. bathrooms<br>
3. sqft_living<br>
4. distance<br></b>
    </div>

In [ ]:
df_num4.describe()

### Categorical variables: Investigating distributions and the raw correlations between the original, minimally processed predictor and the target (price)

#### Original categorical variables: waterfront, view, condition, basement_exists, renovation_done, month

In [ ]:
#Creating a DataFrame with categorical variables

df_cat1 = df_2.filter(['price','waterfront','view','condition','basement_exists','renovation_done','month'], axis=1)


#### Visual investigation of the box plots

In [ ]:
df_cat1

In [ ]:
names=['waterfront','view','condition','basement_exists','renovation_done','month']
for col in names:
    boxen_plot(df_cat1,col)

<div class="alert alert-block alert-info"> Based on the plots, it is quite clear that 'month', 'condition' and 'basement_exists' variables do not affect the price of the properties and can be dropped from the categorical variables<br>
</div>

#### Dumming out variables in the categorical DataFrame

In [ ]:
# Using OheHotEncoder to dummy categorical out. Based on the boxplots above, I am leaving 
# 'month','basement_exists','condition' variables out, they seem not to have too much effect on the target.
# The reason for removing 'waterfront' variable is a tiny porting of all properties in the dataset.

ohe=OneHotEncoder(drop='first')
df_cat_transform=ohe.fit_transform(df_cat1.drop(['price','month','basement_exists','condition','waterfront'], axis=1))
df_cat1_trsfm=pd.DataFrame(df_cat_transform.todense(), 
                          columns=ohe.get_feature_names(['view','renovation_done']))
df_cat1_trsfm.info()

In [ ]:
# Adding a price column and resetting the index 
# Due to the removal of the extreme values in numerical values, some rows have Nan price values; these rows are dropped

df_cat1_trsfm['price']=df_num4['price']

df_cat1_trsfm=df_cat1_trsfm[df_cat1_trsfm.price.notna()]

df_cat1_trsfm=df_cat1_trsfm.reset_index()

df_cat1_trsfm=df_cat1_trsfm.drop('index', axis=1)

df_cat1_trsfm

<div class="alert alert-block alert-success">
    <b>df_cat1_trsfm</b> DataFrame<br><br><b>19811</b> records out of original <b>21597</b> left
    </div><br>
<div class="alert alert-block alert-danger">
    <b>Index reset</b>
    </div><br>

In [ ]:
df_cat1_trsfm.info()

# Model

## Data Modeling


### Baseline model

><i><b>“Everything should be made as simple as possible, but no simpler.”</b></i><div style="text-align: right">Albert Einstein</div>

The chosen baseline model is a model with only one numerical variable, grade.

#### Creating a model

In [ ]:
## Create the formula and  the model

f  = 'price~grade'

model_baseline = smf.ols(f, df_num4).fit()
model_baseline.summary()

In [ ]:
fig = sm.graphics.qqplot(model_baseline.resid,dist=stats.norm,fit=True,line='45')

<div class="alert alert-block alert-info"> The baseline model has a coefficient of determination of 0.452, indicating that roughly 45% of the observations fit the model. F-statistics is very high that indicates that the baseline model is a significant improvement of the  "intercept only model"<br>
    The Skewness and the Kurtosis values indicate non-normal distribution of the target variable<br>
    QQ plot is also indicative of the abnormal distribution of the residuals, especially in the upper Quantile
</div>

### Model 1 (all numerical variables considered significant, see Explore section)

In [ ]:
## Create a formula including the remaining numerical variables

variables_to_include = ' + '.join(df_num4.drop('price',axis=1).columns)

## Create the formula and  the model
f  = "price~" + variables_to_include

model_1 = smf.ols(f, df_num4).fit()
model_1.summary()

#### QQ plot: to access normality of the target variable

In [ ]:
fig = sm.graphics.qqplot(model_1.resid,dist=stats.norm,fit=True,line='45')

<div class="alert alert-block alert-info"> 
The summary of the model above indicates that<br>
    1. All the independent variables coefficients and the intercept value are significant (p-values< 0.05)<br>
    2. The coefficient of determination (R2) is not very high, but it is significantly higher than R2 of the baseline model. It indicates that about 66.8 percent of the observations fall within the regression line <br>
    3. The skew and the Kurtosis values indicate the highly non-normal distribution of the target variable<br>
    4. The high value of JB coefficient also indicates that the data is highly non-normal <br>
</div>

<div class="alert alert-block alert-info"> 
From the model's QQ plot, it is also quite obvious that the 'price' target variable is not normally distributed. A steep swing up indicates that the higher-priced houses are less likely to fit the baseline model and are more spread out. One possible reason might be <b>an unusually large number of outliers in the dataset</b></div>

<div class="alert alert-block alert-danger"><b>There are two potential approaches that can be taken</b><br>
        1. Normalization by either log or square root transformation<br>
        2. Removal of outliers</div>

####  Plot regression results against each regressor: accessing linearity of there relationship with the target and their homoscedasticity 

In [ ]:
for col in (df_num4.drop('price',axis=1).columns):
    fig = sm.graphics.plot_regress_exog(model_1, col, fig=plt.figure(figsize=(12,8)))

<div class="alert alert-block alert-info"> The results indicate that all of the predictors display linear relationship with the target. The distance, sqft_living and bathrooms variables display less  heteroscedasticity than the grade variable does.<br><br></div>
<div class="alert alert-block alert-danger"><b>There might be several appropriate ways to address this issue</b><br>
    1. Log transformation of the target and/or independent variables<br>
    2. Using either Generalized Least Squares or Weighted Least squares<br>
    3. Bootstrapping<br>

### Model 2 (adding categorical variables)

#### Creating a model with all numerical variables and all categorical variables

In [ ]:
df_num4

In [ ]:
df_cat1_trsfm

In [ ]:
# DatyaFrames to concatinate: df_cat1_trsfm AND df_num4
df_num_cat_1 = pd.concat([df_num4, df_cat1_trsfm.drop('price',axis=1)],axis=1)

In [ ]:
# Visualizing multicollinearity in the new dataset
fig, ax = plt.subplots(figsize=(15,10))
sns.heatmap(df_num_cat_1.corr(), cmap="cubehelix", annot=True)

<div class="alert alert-block alert-info">The matrix indicates no strong correlation between the price and any of the categorical variables. However, renovation_done_4 correlation is slightly higher than the rest of the categorical variables. Conceptually this variables indicative of a recent renovation or a newer property.</div>
<div class="alert alert-block alert-danger">There is no high expectations that adding the categorical variables to the mix will significantly improve the model</div>

In [ ]:
df_num_cat_1

In [ ]:
# Create a formula for the numerical variables from the basemodel
# AND the categorical variables from the previous section

variables_to_include = ' + '.join(df_num_cat_1.drop('price',axis=1).columns)

## Create the formula and the model
f  = "price~" + variables_to_include


model_2 = smf.ols(f, df_num_cat_1).fit()
model_2.summary()

In [ ]:
fig = sm.graphics.qqplot(model_2.resid,dist=stats.norm,fit=True,line='45')

<div class="alert alert-block alert-info">The summary indicates a very slight improvement over the previous model, 66.9% versus 66.8% of all of the observations fall within the results of the line formed by the regression equation.<br><br>
    It is also evident that p-values of most of the categorical values are very high, indicating their insignificance in the model. However, because they describe the same feature, I am leaving them in for now<br><br>
The residuals normality did not improve</div>

### Model 3 (preprocessing and removal of outliers)

#### Scaling with Robust Scaler

<div class="alert alert-block alert-info"> 
I ruled out the scaling of the data in this step of the process because it would not change the statistics of the mode, though the correlation coefficients would become more compatible with each other.
<br><br>I am leaving the snippets in the notebook in case I reconsider.
</div>
<div class="alert alert-block alert-danger"><b>The next step is a removal of ourliers</b><br>
</div>

In [ ]:
# Using RobustScaler to scale the data
"""# Using RobustScaler, which transforms the feature vector by subtracting the median and then dividing by the
# interquartile range (%25-75%). It is the most robust to the ourliers

df_for_scalers=df_num_cat_2.copy()

cols=['bathrooms','grade','sqft_living15','distance','waterfront_1','renovation_done_4']
scaler = RobustScaler()
robust_df = scaler.fit_transform(df_for_scalers.drop('price',axis=1))
robust_df = pd.DataFrame(robust_df, columns=cols)

df_num_cat_2.describe()"""

"""robust_df.describe()"""

"""fig, (ax1, ax2) = plt.subplots(ncols = 2, figsize =(20, 5))
ax1.set_title('Before Scaling')
  
sns.kdeplot(df_for_scalers['distance'], ax = ax1, color ='blue')

ax2.set_title('After Robust Scaling')
  
sns.kdeplot(robust_df['distance'], ax = ax2, color ='red')


plt.show()"""

In [ ]:
# Building and summary of the model with scaled data
"""robust_df['price']=df_for_scalers['price']
robust_df.info()


num_cat_var2_robust = ' + '.join(robust_df.drop('price',axis=1).columns)

## Create the formula and the model
f  = "price~" + num_cat_var2_robust


model_num_cat_2_robust = smf.ols(f, robust_df).fit()
model_num_cat_2_robust.summary()

coeffs=model_num_cat_2.params
coeffs.sort_values().round(2)

coeffs=model_num_cat_2_robust.params
coeffs.sort_values().round(2)"""

#### Removal of outliers

##### IQR method

###### Using IQR

In [ ]:
df_num_cat_1

In [ ]:
df_num_cat_2=df_num_cat_1.copy()

# Due to the fact that discrete variables are not suitable for IQR method outlier removal, they are being dropped 
# from the DataFrame. They will be added back to the Dataframe for modeling

df_num_cat_2=df_num_cat_2.drop(['view_1','view_2','view_3','view_4',
                              'renovation_done_1','renovation_done_2','renovation_done_3','renovation_done_4',
                              'grade','bathrooms'], axis=1)

Q1 = df_num_cat_2.quantile(q=.25)
Q3 = df_num_cat_2.quantile(q=.75)
IQR = df_num_cat_2.apply(stats.iqr)

df_num_cat_3 = df_num_cat_2[~((df_num_cat_2 < (Q1-1.5*IQR)) | (df_num_cat_2 > (Q3+1.5*IQR))).any(axis=1)]

df_num_cat_3

In [ ]:
df_num_cat_3['grade']=df_num_cat_1['grade']
df_num_cat_3['bathrooms']=df_num_cat_1['bathrooms']
df_num_cat_3['view_1']=df_num_cat_1['view_1']
df_num_cat_3['view_2']=df_num_cat_1['view_2']
df_num_cat_3['view_3']=df_num_cat_1['view_3']
df_num_cat_3['view_4']=df_num_cat_1['view_4']
df_num_cat_3['renovation_done_1']=df_num_cat_1['renovation_done_1']
df_num_cat_3['renovation_done_2']=df_num_cat_1['renovation_done_2']
df_num_cat_3['renovation_done_3']=df_num_cat_1['renovation_done_3']
df_num_cat_3['renovation_done_4']=df_num_cat_1['renovation_done_4']

In [ ]:
df_num_cat_3.info()

In [ ]:
df_num_cat_3=df_num_cat_3.reset_index()

df_num_cat_3.info()

In [ ]:
df_num_cat_3=df_num_cat_3.drop('index', axis=1)
df_num_cat_3.info()

<div class="alert alert-block alert-success">
    <b>df_num_cat_3</b> DataFrame<br><br><b>18619</b> records out of the original <b>21597</b> left
    </div><br>
<div class="alert alert-block alert-danger">
    <b>Index reset</b>
    </div><br>

###### Building the model

In [ ]:
## Formula is the same, model is for the cleaned DF

variables_to_include_3_1 = ' + '.join(df_num_cat_3.drop('price',axis=1).columns)
f  = "price~" + variables_to_include_3_1


model_3_1 = smf.ols(f, df_num_cat_3).fit()
model_3_1.summary()

In [ ]:
fig = sm.graphics.qqplot(model_3_1.resid,dist=stats.norm,fit=True,line='45')

<div class="alert alert-block alert-info"> While the IQR removal of outliers decreased the R squared of the model, it made the distribution more normal (Skew and Kurtosis values are almost within the normality ranges). This fact is also reflected by the QQ plot of the model residuals. Unfortunately, the Coefficient of determination dropped</div>

##### Z-score method

###### Using Z scores

In [ ]:
df_num_cat_1

In [ ]:
df_num_cat_4=df_num_cat_1.copy()

df_num_cat_4=df_num_cat_4.drop(['view_1','view_2','view_3','view_4',
                              'renovation_done_1','renovation_done_2','renovation_done_3','renovation_done_4',
                              'grade','bathrooms'], axis=1)

df_num_cat_4['z_sqft_living']=stats.zscore(df_num_cat_4['sqft_living'])
df_num_cat_4['z_distance']=stats.zscore(df_num_cat_4['distance'])
df_num_cat_4['z_price']=stats.zscore(df_num_cat_4['price'])

df_num_cat_4

In [ ]:
df_num_cat_5=df_num_cat_4[(abs(df_num_cat_4.z_price) < 3)]

df_num_cat_5=df_num_cat_5[(abs(df_num_cat_5.z_distance) < 3)]

df_num_cat_5=df_num_cat_5[(abs(df_num_cat_5.z_sqft_living) < 3)]
df_num_cat_5

In [ ]:
df_num_cat_5['grade']=df_num_cat_1['grade']
df_num_cat_5['bathrooms']=df_num_cat_1['bathrooms']
df_num_cat_5['view_1']=df_num_cat_1['view_1']
df_num_cat_5['view_2']=df_num_cat_1['view_2']
df_num_cat_5['view_3']=df_num_cat_1['view_3']
df_num_cat_5['view_4']=df_num_cat_1['view_4']
df_num_cat_5['renovation_done_1']=df_num_cat_1['renovation_done_1']
df_num_cat_5['renovation_done_2']=df_num_cat_1['renovation_done_2']
df_num_cat_5['renovation_done_3']=df_num_cat_1['renovation_done_3']
df_num_cat_5['renovation_done_4']=df_num_cat_1['renovation_done_4']

In [ ]:
df_num_cat_5=df_num_cat_5.drop(['z_sqft_living','z_distance','z_price'], axis=1)

In [ ]:
df_num_cat_5=df_num_cat_5.reset_index()
df_num_cat_5=df_num_cat_5.drop('index', axis=1)
df_num_cat_5.info()

<div class="alert alert-block alert-success">
    <b>df_num_cat_5</b> DataFrame<br><br><b>19261</b> records out of the original <b>21597</b> left
    </div><br>
<div class="alert alert-block alert-danger">
    <b>Index reset</b>
    </div><br>

##### Building the model

In [ ]:
## Formula is the same, model is for the cleaned DF

variables_to_include_3_2 = ' + '.join(df_num_cat_5.drop('price',axis=1).columns)
f  = "price~" + variables_to_include_3_2


model_3_2 = smf.ols(f, df_num_cat_5).fit()
model_3_2.summary()

In [ ]:
fig = sm.graphics.qqplot(model_3_2.resid,dist=stats.norm,fit=True,line='45')

<div class="alert alert-block alert-info"> 
The R squared of the model is 0.654 and F-statistics is higher than for the previous model
    <br>The IQR method of outliers removal made the residual distribution more normal than Z-score method due to the former having more strict criteria. The decision is to use the dataset compiled after Z-score outlier removal. 
<br>
</div>
<div class="alert alert-block alert-danger"><b>The next step is Log transformation of the target variable</b><br>
</div>

### Model 4 (Using log and square root transformations on the target variable)

#### Log Transformation

In [ ]:
df_num_cat_5.info()

In [ ]:
# Log transform
df_num_cat_5_log=df_num_cat_5.copy()

df_num_cat_5_log['log_price'] = df_num_cat_5['price'].map(lambda x: np.log(x))
df_num_cat_5_log

In [ ]:
df_num_cat_5_log.info()

In [ ]:
# Histogram of log_price and price
continuous=['price','log_price']
for col in continuous:
    fig, ax =plt.subplots(figsize=(5, 5))
    distribution(df_num_cat_5_log[col])


<div class="alert alert-block alert-info"> 
The transformation worked well, improving the normality of the 'price' variable. Log_price distribution looks more symmetrical. Skewness improved dramatically (from 1.18 to -0.01, 0 being perfectly symmetrical)
<br><br>
 Kurtosis value decreased, making the curve more Mesokurtic (close to a Gaussian curve). It is an expected effect of log transform.
</div>
<div class="alert alert-block alert-danger"><b>The next step is test a square root transformation.</b><br>
</div>

In [ ]:
# Square transform
df_num_cat_5_sqrt=df_num_cat_5.copy()

df_num_cat_5_sqrt['sqrt_price'] = df_num_cat_5['price'].map(lambda x: np.sqrt(x))
df_num_cat_5_sqrt

In [ ]:
# Histogram of log_price and price
continuous=['price','sqrt_price']
for col in continuous:
    fig, ax =plt.subplots(figsize=(5, 5))
    distribution(df_num_cat_5_sqrt[col])


<div class="alert alert-block alert-info"> 
The square root transformation also worked well in improving the normality of the 'price' variable. 
sqrt_price distribution looks more symmetrical. Skewness improved dramatically (from 1.18 to -0.59, 0 being perfectly symmetrical)
<br><br>
 However, both of the parameters are worse that the parameters of log_price distribution.
</div>
<div class="alert alert-block alert-danger"><b>The next step is create two separate models and to see if the transformations made a difference</b><br>
</div>

#### Model using log transformed target variable

In [ ]:
## Formula is the same, model is for the cleaned DF

variables_to_include_4_1 = ' + '.join(df_num_cat_5_log.drop(['price','log_price'],axis=1).columns)
f  = "log_price~" + variables_to_include_4_1


model_4_1 = smf.ols(f, df_num_cat_5_log).fit()
model_4_1.summary()

#### Model using square root transformed target variable

In [ ]:
## Formula is the same, model is for the cleaned DF

variables_to_include_4_2 = ' + '.join(df_num_cat_5_sqrt.drop(['price','sqrt_price'],axis=1).columns)
f  = "sqrt_price~" + variables_to_include_4_2


model_4_2= smf.ols(f, df_num_cat_5_sqrt).fit()
model_4_2.summary()

In [ ]:
fig = sm.graphics.qqplot(model_4_1.resid,dist=stats.norm,fit=True,line='45')

In [ ]:
fig = sm.graphics.qqplot(model_4_2.resid,dist=stats.norm,fit=True,line='45')

<div class="alert alert-block alert-info"> 
Both models have improved the R squared and the F-statistics of the previous models. The residuals of both models display a close-to-normal distribution. Log transformation helped improve the upper part of the distribution, while square root transformation worked better in the lower part of the distribution. 
<br><br>
R squared of the square root transformation-based model is slightly higher, while its kurtosis value is slightly worse than the kurtosis value of the log-transformed price model. The decision is to use the log-transformed target variable.
</div>
<div class="alert alert-block alert-danger"><b>The next step is to remove unnecessary categorical variables, scale the remaining variables, and built the last model with coefficients in the regression model, which are easy to compare and interpret</b><br>
</div>


In [ ]:
# Removing all categorical values with high p-values and renaming the renovation_done_4
# to a more prominent name, reflective of the definition of the field (see Categorical variables subsection 
# in Exploring distributions and correlations section)

df_num_cat_5_log_cropped=df_num_cat_5_log.copy()
df_num_cat_5_log_cropped=df_num_cat_5_log_cropped.drop(['view_1','view_2','view_3','view_4','renovation_done_1',
                                                'renovation_done_2','renovation_done_3'], axis=1)
df_num_cat_5_log_cropped.rename(columns={'renovation_done_4': 'recent_renovation_new'}, inplace=True)

In [ ]:
df_num_cat_5_log_cropped

In [ ]:
# Standardizing the independent variables. I decided to do it manually due to a better control of the utput


df_num_cat_5_log_cropped_st=df_num_cat_5_log_cropped.copy()


lv_min=df_num_cat_5_log_cropped_st.sqft_living.min()
lv_range=df_num_cat_5_log_cropped_st.sqft_living.max()-df_num_cat_5_log_cropped_st.sqft_living.min()
distance_range=df_num_cat_5_log_cropped_st.distance.max()-df_num_cat_5_log_cropped_st.distance.min()
distance_min=df_num_cat_5_log_cropped_st.distance.min()
bathroom_range=df_num_cat_5_log_cropped_st.bathrooms.max()-df_num_cat_5_log_cropped_st.bathrooms.min()
bathroom_min=df_num_cat_5_log_cropped_st.bathrooms.min()
grade_range=df_num_cat_5_log_cropped_st.grade.max()-df_num_cat_5_log_cropped_st.grade.min()
grade_min=df_num_cat_5_log_cropped_st.grade.min()


df_num_cat_5_log_cropped_st['sqft_living_st']=df_num_cat_5_log_cropped_st.apply(lambda row: round((row.sqft_living-lv_min)/lv_range,3), axis=1)
df_num_cat_5_log_cropped_st['distance_st']=df_num_cat_5_log_cropped_st.apply(lambda row: round((row.distance-distance_min)/distance_range,3), axis=1)
df_num_cat_5_log_cropped_st['bathrooms_st']=df_num_cat_5_log_cropped_st.apply(lambda row: round((row.bathrooms-bathroom_min)/bathroom_range,3), axis=1)
df_num_cat_5_log_cropped_st['grade_st']=df_num_cat_5_log_cropped_st.apply(lambda row: (row.grade-grade_min)/grade_range, axis=1)
#df_num_cat_5_sqrt_cropped_st['recent_renovation_new_str']=df_num_cat_5_sqrt_cropped_st['recent_renovation_new'].astype('str')

df_num_cat_5_log_cropped_st

In [ ]:
## Formula is the same, model is for the cleaned DF

variables_to_include_4_3 = ' + '.join(df_num_cat_5_log_cropped_st.drop(
    ['price','log_price','sqft_living','distance','grade','bathrooms'],axis=1).columns)
f  = "log_price~" + variables_to_include_4_3
print(f)
model_4_3= smf.ols(f, df_num_cat_5_log_cropped_st).fit()
model_4_3.summary()

In [ ]:
fig = sm.graphics.qqplot(model_4_3.resid,dist=stats.norm,fit=True,line='45')

# Removing variable to simplify the display (categorical variable with only 2 values does not have
# much use when plotted by regress_exog)
# print(df_num_cat_5_log_cropped_st.drop(['price','log_price','recent_renovation_new',
#                                        'sqft_living', 'distance', 'grade', 'bathrooms'],axis=1).columns)


for col in (df_num_cat_5_log_cropped_st.drop(['price','log_price','recent_renovation_new',
                                        'sqft_living', 'distance', 'grade', 'bathrooms'],axis=1).columns):
    fig = sm.graphics.plot_regress_exog(model_4_3, col, fig=plt.figure(figsize=(12,8)))

<div class="alert alert-block alert-info"> 

The final linear regression model of a log of the price variable versus grade, bathrooms, distance from the center of the city, sqft_living space, and the indicator if a house has been renovated recently or a newer house has a Coefficient of Determination of 0.663. It is indicative of the fact that 66.3% of the sold properties fall within the results of the line formed by the regression equation. F-statistics displays the high value and the overall p-value much lower than the confidence interval 
<br><br>
The independent variables used in the equation display a clear linear relationship with the target and homoscedasticity.
</div>

<div class="alert alert-block alert-danger"><b>The next step is to validate the model by using training and test datasets</b><br>

</div>

### Train the model

In [ ]:
# Define X and y
y=df_num_cat_5_log_cropped_st[['log_price']]
X=df_num_cat_5_log_cropped_st.drop(['price','log_price','recent_renovation_new',
                                        'sqft_living', 'distance', 'grade', 'bathrooms'], axis=1)

In [ ]:
y

In [ ]:
X

In [ ]:
# Split the data into training and test sets. Use the default split size
# X_train, X_test, y_train, y_test = train_test_split(X, y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=100)

print(len(X_train), len(X_test), len(y_train), len(y_test))

In [ ]:
# Fit the model to train data
linreg = LinearRegression()
model=linreg.fit(X_train, y_train)

# Calculate predictions on training and test sets for y_hat
y_hat_train=linreg.predict(X_train)
y_hat_test=linreg.predict(X_test)

# Calculate residuals
train_residuals=y_hat_train-y_train
test_residuals=y_hat_test-y_test

# Calculate training and test RMSE
train_mse = mean_squared_error(y_train, y_hat_train, squared=False)
test_mse = mean_squared_error(y_test, y_hat_test, squared=False)
print('Train MSE:', round(train_mse,3))
print('Test MSE:', round(test_mse,3))

In [ ]:
print(model.coef_, model.intercept_, model.score(X_test, y_test))

### Validation

In [ ]:
price_target=df_num_cat_5_log_cropped_st[['log_price']]
price_predictors= df_num_cat_5_log_cropped_st.drop(['price','log_price','recent_renovation_new',
                                        'sqft_living', 'distance', 'grade', 'bathrooms'], axis=1)

In [ ]:
metrics.r2_score(price_target, linreg.predict(price_predictors))

In [ ]:
metrics.mean_absolute_error(price_target, linreg.predict(price_predictors))

In [ ]:
mean_squared_error(price_target, linreg.predict(price_predictors))

>Train MSE: 0.272<br>
Test MSE: 0.272<br>
Are equal down to the third decimal digit indicating a good agreement between the training and the test sets<br><br>
<b>R</b> squared for for the prediction on the full dataset is the same as in model_4_3: 0.663<br>
<b>Mean Absolute Error</b> for the prediction on the full dataset is 0.213 which is not great but acceptable. The best possible theoretical value is 0.<br>
<b>Mean Squared Error</b> for the prediction on the full dataset is 0.074. The best possible theoretical value is 0.


In [ ]:
# Not sure if the results of this function are indicative of anything, but leaving it here for a possible future use
"""mse, bias, var = bias_variance_decomp(model, X_train.values, y_train.values.flatten(), 
                                      X_test.values, y_test.values, num_rounds=200, random_seed=1, loss='mse')
# summarize results
print('MSE: %.3f' % mse)
print('Bias: %.3f' % bias)
print('Variance: %.3f' % var)"""

# iNterpret

In [ ]:
%%latex

\begin{align}\ln\left( {Price} \right) = 12.366 + 1.265 \cdot \left( grade \right) + 
1.080 \cdot \left( sqft\_living \right) - 0.989 \cdot \left( distance \right) 
+ 0.060 \cdot \left( bathrooms \right) - 0.035 \cdot \left( recent\_renovation\_new \right) \end{align}

The final model has a reasonable predictive ability tested in the final step of the model validation. MSE, MAE, and R2 score along with the model p-values for all predictors indicate a good fit.<br><br>
The most influential predictor is a **building grade**, following by a **living space footage**. Both factors are **positively correlated** with the price of the property. Both factors are within property owners' control when they are renovating their houses.<br>A **distance** from the center of the city is **negatively correlated** with the price of property, meaning the further away a property is, the lower is the price. It is not a controllable variable but is helpful for home buyers if the living space and the number of bedrooms/bathrooms are important.<br> A **number of bathrooms** has a **positive effect** on the price of a property, but it is not as strong as the first two factors. This fact indicates that the convenience of having multiple bathrooms is essential for potential buyers and should be taken into account when owners are planning a renovation.<br>
The last predictor in the model is an indicator of whether a property **has been renovated recently or a new construction**. It is very **weakly negatively correlated** with the price variable. The negative correlation (reduction of the price) might be related to the following factors: newer properties, on the average, are of less building quality.<br>
**The intercept** of the model is a **bias** of the model and can be interpreted as an offset of the model due to other factors not taken into account for various reasons.

# Conclusions and Recommendation

**Recommendations to property owners planning a renovation to their properties:**
* Increase the living space of your property
* Do the renovation with higher building quality
* Consider adding a bathroom

**Recommendations to potential buyers:**
* Look for properties further away from the city center to make the best out of your property buying budget
* Properties in some zipcodes of the city are more affordable than others at the same distance from the city center
* Properties in some zipcodes of the city are more affordable than others with a better view, more considerable property lots, and with older houses of better quality construction if these factors are essential to a buyer

**Limitations of the model:**
* The original dataset does not include other important factors, and therefore the model is biased
* Multiple linear regression models, while easily interpretable, are limited in their predictive ability
* Some variables in the dataset are strongly correlated with each other, and that affect the predictive power of the model

**Suggestion for future improvements**:
* Add variables to the original dataset like kitchen renovation, average commute time, crime index, average nearby public school quality, etc.
* Update the dataset with more current data

# Appendix

### Visualization

In [ ]:
df_test=X_test.copy()
df_test['log_price']=y_test['log_price']
df_test['recent_renovation_new']=df_num_cat_5_log_cropped_st['recent_renovation_new']
df_test['sqft_living']=df_num_cat_5_log_cropped_st['sqft_living']
df_test['distance']=df_num_cat_5_log_cropped_st['distance']
df_test['grade']=df_num_cat_5_log_cropped_st['grade']
df_test['bathrooms']=df_num_cat_5_log_cropped_st['bathrooms']
df_test['price']=df_test.apply(lambda row: math.exp(row.log_price), axis=1)
df_test['recent_renovation_new_str']=df_test['recent_renovation_new'].astype('str')
#df_test=df_test.reset_index(drop='index')
df_test

In [ ]:
# Regplots for all four variables

sns.set_style("darkgrid", {"axes.facecolor": ".6"})
fig, axes = plt.subplots(figsize=(20,20), ncols=2, nrows=2)

g1=sns.regplot(data=df_test, x="sqft_living_st", y="price", color="#003300", fit_reg=True,
               ax=axes[0,0], line_kws={"color": "red", "lw":5});
g2=sns.regplot(data=df_test, x="distance_st", y="price", color="#000066", 
               ax=axes[0,1], line_kws={"color": "red", "lw":5});
g3=sns.regplot(data=df_test, x="bathrooms_st", y="price", color="#009900", 
               ax=axes[1,0], line_kws={"color": "red", "lw":5});
g4=sns.regplot(data=df_test, x="grade_st", y="price", color="#0000ff", 
               ax=axes[1,1], line_kws={"color": "red", "lw":5});

axes[0,0].set_title("Prive vs Living Space", fontsize=26);
axes[0,0].set_ylabel('Price', fontsize=20)
axes[0,0].set_xlabel('Living Footage, adjusted', fontsize=20)
axes[0,0].set_xlim(-0.01, 1.0)
ylabels = ['{:,.1f}'.format(x) + 'M' for x in g1.get_yticks()/1000000]
axes[0,0].set_yticklabels(ylabels, size=15)
axes[0,0].grid(color='lightgrey')

axes[0,1].set_title("Price vs Distance from City Center", fontsize=26);
axes[0,1].set_ylabel('Price', fontsize=20)
axes[0,1].set_xlabel('Distance, adjusted', fontsize=20)
axes[0,1].set_xlim(-0.01, 1.0)
ylabels = ['{:,.1f}'.format(x) + 'M' for x in g2.get_yticks()/1000000]
axes[0,1].set_yticklabels(ylabels, size=15)
axes[0,1].grid(color='lightgrey')

axes[1,0].set_title("Price vs Number of Bathrooms", fontsize=26);
axes[1,0].set_ylabel('Price', fontsize=20)
axes[1,0].set_xlabel('Number of Bathrooms, adjusted', fontsize=20)
axes[1,0].set_xlim(-0.01, 1.0)
ylabels = ['{:,.1f}'.format(x) + 'M' for x in g3.get_yticks()/1000000]
axes[1,0].set_yticklabels(ylabels, size=15)
axes[1,0].grid(color='lightgrey')

axes[1,1].set_title("Price vs Building Grade", fontsize=26);
axes[1,1].set_ylabel('Price', fontsize=20)
axes[1,1].set_xlabel('Building Grade, adjusted', fontsize=20)
axes[1,1].set_xlim(-0.01, 1.0)
ylabels = ['{:,.1f}'.format(x) + 'M' for x in g4.get_yticks()/1000000]
axes[1,1].set_yticklabels(ylabels, size=15)
axes[1,1].grid(color='lightgrey')


plt.suptitle("Regression plots of Price vs Four Independent Variables", size=30, c="Blue")
plt.tight_layout(pad=3)


In [ ]:
fig = px.scatter(df_test, x='sqft_living', y='price',  trendline='lowess', trendline_color_override='blue',
                 color='grade', size='distance',width=1000, height=800, size_max=20, 
                 color_continuous_scale=px.colors.sequential.Blackbody_r,
                 labels={
                     "price": "Price",
                     "sqft_living": "Living Space (sq ft)",
                     "grade": "Building Grade"
                  },
                title="Correlation: Property Price vs Living Space Footage",  template = "plotly_dark")

fig.update_traces(marker=dict(
            line=dict(
                color='coral',
                width=0.5
            )))
fig.update_layout(
    font_family="Arial",
    font_size=18,
    font_color="white",
    title_font_family="Arial",
    title_font_color="white",

)
fig.update_layout(
    title={
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top',
})


fig.show()

In [ ]:

fig = px.scatter(df_test, x='bathrooms', y='price', color_continuous_scale=px.colors.sequential.Blackbody_r,
                 size='sqft_living',size_max=20,
                 trendline='lowess', trendline_color_override='blue', color='grade',
                 width=1000, height=800, labels={
                     "price": "Price",
                     "bathrooms": "Number of bathrooms",
                     "grade": "Building Grade"
                 },
                title="Correlation: Property Price vs Number of Bathrooms",
                 template = "plotly_dark")


fig.update_traces(marker=dict(
            line=dict(
                color='coral',
                width=0.5
            )))
fig.update_layout(
    font_family="Arial",
    font_size=18,
    font_color="white",
    title_font_family="Arial",
    title_font_color="white",

)

fig.update_layout(
    title={
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top',
})


fig.show()

In [ ]:
fig = px.scatter(df_test, x='distance', y='price',  trendline='lowess', trendline_color_override='blue',
                 color='grade', size='bathrooms',width=1000, height=800, size_max=20, 
                 color_continuous_scale=px.colors.sequential.Blackbody_r,
                 labels={
                     "price": "Price",
                     "distance": "Distance",
                     "grade": "Building Grade"
                  },
                title="Correlation: Property Price vs Distance from the City Center",  template = "plotly_dark")

fig.update_traces(marker=dict(
            line=dict(
                color='coral',
                width=0.5
            )))
fig.update_layout(
    font_family="Arial",
    font_size=18,
    font_color="white",
    title_font_family="Arial",
    title_font_color="white",

)
fig.update_layout(
    title={
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top',
})


fig.show()

In [ ]:
fig = px.scatter(df_test, x='sqft_living', y='price',  trendline='ols', trendline_color_override='yellow',
                 color='recent_renovation_new_str', width=1000, height=800, size_max=20, 
                 labels={
                     "price": "Price",
                     "sqft_living": "Living Space (sq ft)",
                     "recent_renovation_new_str": "Newer(1)/Older(0)"
                  },
                title="Correlation: Property Price vs Living Space Footage of newer vs older properties",
                 template = "plotly_dark")

fig.update_traces(marker=dict(
            line=dict(
                color='coral',
                width=0.5
            )))
fig.update_layout(
    font_family="Arial",
    font_size=18,
    font_color="white",
    title_font_family="Arial",
    title_font_color="white",

)
fig.update_layout(
    title={
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top',
})


fig.show()

In [ ]:
fig = px.scatter_3d(df_test, x='bathrooms', z='grade', y='sqft_living',
              color='price', size='distance', size_max=50, opacity=1, width=1000, height=800,
                   color_continuous_scale=px.colors.sequential.Blackbody_r, 
                   labels={
                     "bathrooms": "Number of Bathrooms",
                     "sqft_living": "Living Space (sq ft)",
                     "grade": "Grade",
                       "price": "Price"
                  },
                title="3D plot: Living Space Footage, Number of Bathrooms and Grade of Sold Properties",
                   template = "plotly_dark")

fig.update_traces(marker=dict(
            line=dict(
                color='coral',
                width=0.5
            )))
fig.update_layout(
    font_family="Arial",
    font_size=16,
    font_color="white",
    title_font_family="Arial",
    title_font_color="white"
)
fig.update_layout(
    title={
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top',
})


fig.show()

In [ ]:
df_cut_off=df.copy()
df_cut_off=df_cut_off[(df_cut_off['price']<1500000)]

fig = px.scatter_mapbox(df_cut_off, lat="lat", lon="long", color="price", 
                  color_continuous_scale=px.colors.sequential.Plasma, zoom=10, 
                        mapbox_style='open-street-map', width=900, height=900,
                        title="Properties Sold in King County in 2014-2015",
                   template = "plotly_dark")

fig.update_layout(
    font_family="Arial",
    font_size=20,
    font_color="white",
    title_font_family="Arial",
    title_font_color="white"
)
fig.update_layout(
    title={
        'y':0.98,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top',
})


fig.show()

In [ ]:
df_zipcode_viz=df.groupby('zipcode').mean()

df_zipcode_viz=df_zipcode_viz.reset_index()

df_zipcode_viz=df_zipcode_viz.drop(['id','sqft_above','sqft_basement','yr_renovated','lat','long'], axis=1)
df_zipcode_viz

In [ ]:
KC_zip_json=json.load(open('data/wa_washington_zip_codes_geo.min.json', 'r'))


In [ ]:
map_choropleth_zip(df_zipcode_viz, 'price', "Average Prices of Sold Properties per Zipcode (King County, 2014-2015)", 
                   "Price")

In [ ]:
map_choropleth_zip(df_zipcode_viz, 'sqft_lot', "Average Lot Size of Sold Properties per Zipcode (King County, 2014-2015)",
                   "Lot size (sq ft)")

In [ ]:
map_choropleth_zip(df_zipcode_viz, 'yr_built', "Average Year Built of Sold Properties per Zipcode (King County, 2014-2015)", 
                   "Year Built")

In [ ]:
map_choropleth_zip(df_zipcode_viz, 'view', "Average View Category of Sold Properties per Zipcode (King County, 2014-2015)", 
                   "View Category")

In [ ]:
map_choropleth_zip(df_zipcode_viz, 'sqft_living', "Average Living Space of Sold Properties per Zipcode (King County, 2014-2015)", 
                   "Living Space (sq ft) ")